In [1]:
!pip install transformers

     |████████████████████████████████| 1.4MB 19.0MB/s 
     |████████████████████████████████| 2.9MB 50.1MB/s 
     |████████████████████████████████| 890kB 55.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b1704cb46cf6293e08352374440e95ec5d61843787c7ba36e4b532561babfa3a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

2020-12-11 12:56:24.564558: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [2]:
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

In [3]:
!pip install jsonlines
import jsonlines
from sklearn.model_selection import train_test_split
!pip install python-levenshtein
import Levenshtein as Lev
import torch
from torch.utils.data import Dataset as TorchDataset
from typing import Sequence, Dict, Any, List
import json

     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144796 sha256=de06a34881165623e34447bf0458b09a5b3c5016fa190f30502c2656f3a4ecc7
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('trec')

     |████████████████████████████████| 163kB 16.7MB/s 
     |████████████████████████████████| 17.7MB 183kB/s 
     |████████████████████████████████| 245kB 55.0MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


Using custom data configuration default


Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48. Subsequent calls will reuse this data.


## **We use pre-trained MLM: Bert for masked LM from the huggingface transformers**

In [6]:
from transformers import BertTokenizerFast, BertForMaskedLM
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [7]:
from transformers import AutoTokenizer
dataset = load_dataset('trec', split='test')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

Using custom data configuration default
Reusing dataset trec (/root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48)


In [8]:
dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'label-coarse', 'label-fine', 'text', 'token_type_ids'],
    num_rows: 500
})

In [9]:
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label-coarse'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
next(iter(dataloader))

/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  2129,  2521,  ...,     0,     0,     0],
         [  101,  2054,  2221,  ...,     0,     0,     0],
         [  101,  2040,  2001,  ...,     0,     0,     0],
         ...,
         [  101,  2054, 14130,  ...,     0,     0,     0],
         [  101,  2054,  2003,  ...,     0,     0,     0],
         [  101,  2073,  2024,  ...,     0,     0,     0]]),
 'label-coarse': tensor([4, 5, 3, 0, 4, 4, 3, 1, 0, 0, 5, 3, 4, 3, 4, 4, 1, 3, 0, 4, 3, 0, 5, 0,
         0, 3, 0, 5, 5, 5, 4, 5]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]

In [10]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [12]:
import numpy as np
import random 

def mask_tokens(batch, tokenizer):
    batch_masked, masked_inds = [], []
    for b in batch:
        inds_to_mask = np.random.choice(torch.arange(1, (b[b!=0].shape[0] -1)), size=1)
        masked_inds.append(inds_to_mask)
        b_new = b.clone()
        b_new[inds_to_mask] = tokenizer.mask_token_id
        batch_masked.append(b_new)
    return torch.stack(batch_masked), masked_inds

In [17]:
all_logits = []
all_input = []
all_attent_masks = []
counter = 0
for batch in tqdm(dataloader):
    with torch.no_grad():
        if counter < 5:
            b_input_ids = batch['input_ids']
            b_input_mask = batch['attention_mask'].to(device)
            
            b_masked, masked_inds = mask_tokens(b_input_ids, tokenizer)
            logits = model(b_masked.to(device), attention_mask=b_input_mask)
            all_logits.append(logits[0])
            all_attent_masks.append(b_input_mask)
            all_input.append(b_input_ids)
            counter = counter + 1
        else:
            break


In [18]:
generated_batch_sentence = []
original_batch_sentence = []
for logits, inputs, attention_mask in zip(all_logits, all_input, all_attent_masks):
    for i in range(32):
        generated = (torch.argmax(torch.softmax(logits[i], axis=1), axis=1) * attention_mask[i])
        original = inputs[i]
        original_batch_sentence.append(tokenizer.decode(original[(original!=0) & (original!=101) & (original!=102)]))
        generated_batch_sentence.append(tokenizer.decode(generated[(generated != 0) & (generated!=1012)]))
        
sentences = generated_batch_sentence + original_batch_sentence

In [24]:
for i in range(32):
    generated = (torch.argmax(torch.softmax(all_logits[1][i], axis=1), axis=1) * all_attent_masks[1][i])
    original = all_input[1][i]
    print('Original:')
    print(tokenizer.decode(original[(original!=0) & (original!=101) & (original!=102)]))
    print('Generated:')
    print(tokenizer.decode(generated[(generated != 0) & (generated!=1012)]))
    print('-'*40)


Original:
what are invertebrates?
Generated:
what about invertebrates?
----------------------------------------
Original:
what is the temperature at the center of the earth?
Generated:
what is the temperature at the center of the earth?
----------------------------------------
Original:
when did john f. kennedy get elected as president?
Generated:
when did john f kennedy get elected as president?
----------------------------------------
Original:
how old was elvis presley when he died?
Generated:
how old was elvis, when he died?
----------------------------------------
Original:
where is the orinoco river?
Generated:
where is the orinoco river?
----------------------------------------
Original:
how far is the service line from the net in tennis?
Generated:
how far is the service line from the net in question?
----------------------------------------
Original:
how much fiber should you have per day?
Generated:
how much fiber do you have per day?
----------------------------------------


In [25]:
def calculate_wer(sequence_a: str, sequence_b: str) -> int:
    # taken from https://github.com/SeanNaren/deepspeech.pytorch/blob/master/decoder.py
    b = set(sequence_a.split() + sequence_b.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in sequence_a.split()]
    w2 = [chr(word2char[w]) for w in sequence_b.split()]

    return Lev.distance(''.join(w1), ''.join(w2))

In [29]:
class SequenceModifier:
    # taken from https://github.com/fursovia/dilma/blob/ef582e73a1cfce5f6bd753fb08b588f12f5cde8f/adat/utils.py#L103
    def __init__(
            self,
            vocab: List[str],
            remove_prob: float = 0.05,
            add_prob: float = 0.05,
            replace_prob: float = 0.1
    ) -> None:
        assert sum([remove_prob, add_prob, replace_prob]) > 0.0
        self.vocab = vocab
        self.remove_prob = remove_prob
        self.add_prob = add_prob
        self.replace_prob = replace_prob

    def remove_token(self, sequence: List[str]) -> List[str]:
        samples = np.random.binomial(n=1, p=self.remove_prob, size=len(sequence))
        sequence = [token for i, token in enumerate(sequence) if not samples[i]]
        return sequence

    def replace_token(self, sequence: List[str]) -> List[str]:
        samples = np.random.binomial(n=1, p=self.replace_prob, size=len(sequence))
        new_sequence = [random.choice(self.vocab) if samples[i] else sequence[i] for i in range(len(sequence))]
        return new_sequence

    def add_token(self, sequence: List[str]) -> List[str]:
        new_sequence = sequence + [
            random.choice(self.vocab)
            for _ in range(np.random.binomial(len(sequence), self.add_prob))
        ]
        return new_sequence

    def __call__(self, sequence: str) -> str:
        splitted_sequence = sequence.split()
        if len(splitted_sequence) > 1 and self.remove_prob:
            splitted_sequence = self.remove_token(splitted_sequence)

        if self.replace_prob:
            splitted_sequence = self.replace_token(splitted_sequence)

        if self.add_prob:
            splitted_sequence = self.add_token(splitted_sequence)
        return " ".join(splitted_sequence)

In [41]:
dataset_dl = []
non_adversarial_indexes = np.random.randint(0, len(sentences), size=(13000, 2))
for id1, id2 in tqdm(non_adversarial_indexes):
    tr1 = sentences[id1].strip()
    tr2 = sentences[id2].strip()
    
    dist = calculate_wer(tr1, tr2)
    ex = {"seq_a": tr1, "seq_b": tr2, "dist": dist}
    dataset_dl.append(ex)

In [42]:
len(dataset_dl)

13000

In [43]:
vocab = []
for seq in sentences:
    vocab.extend(seq.split())

vocab = list(set(vocab))
NUM_SMALL_CHANGES = 3
NUM_SEQUENTIAL_CHANGES = 10

REMOVE_PROB = 0.0
ADD_PROB = 0.0
REPLACE_PROB = 1.0

modifier = SequenceModifier(
    vocab,
    remove_prob = REMOVE_PROB,
    add_prob = ADD_PROB,
    replace_prob = REPLACE_PROB
)
adversarial_indexes = np.random.randint(0, len(sentences), size=(400, ))
for idx in tqdm(adversarial_indexes):
    tr1 = sentences[idx].strip()

    for _ in range(NUM_SMALL_CHANGES):
        tr2 = modifier(tr1).strip()
        dist = calculate_wer(tr1, tr2)
        ex = {"seq_a": tr1, "seq_b": tr2, "dist": dist}
        dataset_dl.append(ex)

    tr2 = tr1
    num_changes = NUM_SEQUENTIAL_CHANGES
    if num_changes > len(tr1.split()):
        num_changes = len(tr1.split())
    for _ in range(num_changes):
        position_idx = random.randint(0, len(tr1.split()) - 1)
        replace_with = random.choice(vocab)
        tr2 = tr2.split()
        tr2[position_idx] = replace_with
        tr2 = " ".join(tr2)

        dist = calculate_wer(tr1, tr2)
        ex = {"seq_a": tr1, "seq_b": tr2, "dist": dist}
        dataset_dl.append(ex)

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
train_path = "/content/gdrive/My Drive/train_dl.json"
test_path = "/content/gdrive/My Drive/test_dl.json"

In [46]:
train, test = train_test_split(dataset_dl, test_size=0.1)

In [47]:
with jsonlines.open(train_path, "w") as writer:
    for ex in train:
        writer.write(ex)

with jsonlines.open(test_path, "w") as writer:
    for ex in test:
        writer.write(ex)